In [1]:
!pip3 install sklearn

In [2]:
# import required libraries
import sys
sys.path.insert(0, '../../../BERT-FAQ/')

from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import ndcg_score
from shared.utils import load_from_json
from metric import NDCG
from tqdm import tqdm
import pandas as pd
import numpy as np
import os.path
import math

In [3]:
# define functions to compute NDCG@, P@, MAP 

def compute_map(result_filepath):
    query_results = load_from_json(result_filepath)

    sum_ap = 0
    num_queries = 0
    for result in query_results:
        query_string = result['query_string']
        topk_results = result['rerank_preds']

        labels = []
        reranks = []

        for topk in topk_results:
            labels.append(topk['label'])
            reranks.append(topk['score'])

        true_relevance = np.array(labels)
        scores = np.array(reranks)

        ap = 0
        all_zeros = not np.any(labels)
        if labels and reranks and not all_zeros:
            ap = average_precision_score(true_relevance, scores)

        sum_ap = sum_ap + ap
        num_queries = num_queries + 1

    return float(sum_ap / num_queries)

def compute_prec(result_filepath, k):
    query_results = load_from_json(result_filepath)

    sum_prec = 0
    num_queries = 0
    for result in query_results:
        query_string = result['query_string']
        topk_results = result['rerank_preds']

        labels = []
        reranks = []

        for topk in topk_results[:k]:
            labels.append(topk['label'])
            reranks.append(topk['score'])

        true_relevance = np.array(labels)
        scores = np.array(reranks)

        prec = 0
        all_zeros = not np.any(labels)
        if labels and reranks and not all_zeros:
            prec = sum(true_relevance) / len(true_relevance)

        sum_prec = sum_prec + prec
        num_queries = num_queries + 1

    return float(sum_prec / num_queries)

def compute_ndcg(result_filepath, k):
    query_results = load_from_json(result_filepath)

    sum_ndcg = 0
    num_queries = 0
    for result in query_results:
        query_string = result['query_string']
        topk_results = result['rerank_preds']

        labels = []
        reranks = []

        for topk in topk_results[:k]:
            labels.append(topk['label'])
            reranks.append(topk['score'])

        true_relevance = np.asarray([labels])
        scores = np.asarray([reranks])

        ndcg = 0
        if labels and reranks:
            ndcg = ndcg_score(true_relevance, scores)

        sum_ndcg = sum_ndcg + ndcg
        num_queries = num_queries + 1

    return float(sum_ndcg / num_queries)

In [4]:
# define function to generate evaluation DataFrame

def get_evaluation_df(rank_results_filepath):
    output = dict()
    metric_name = []

    i = 0

    rankers = ["unsupervised", "supervised"] 
    loss_type = ["triplet", "softmax"]
    query_types = ["faq", "user_query"]
    neg_types = ["simple", "hard"]

    topk = [3, 5, 10]

    for ranker in rankers:

        # Compute metrics for the unsupervised method
        if ranker == "unsupervised":
            file_path = rank_results_filepath + "/" + ranker

            method = ranker

            answer_metric = []
            question_metric = []
            question_answer_metric = []
            question_answer_concat_metric = []

            i += 1
            for k in topk:
                # compute NDCG@k
                if i == 1:
                    metric_name.append("ndcg@" + str(k))

                answer_metric.append(compute_ndcg(file_path + "/es_query_by_answer.json", k))
                question_metric.append(compute_ndcg(file_path + "/es_query_by_question.json", k))
                question_answer_metric.append(compute_ndcg(file_path + "/es_query_by_question_answer.json", k))
                question_answer_concat_metric.append(compute_ndcg(file_path + "/es_query_by_question_answer_concat.json", k))

                # compute P@k
                if i == 1:
                    metric_name.append("prec@" + str(k))

                answer_metric.append(compute_prec(file_path + "/es_query_by_answer.json", k))
                question_metric.append(compute_prec(file_path + "/es_query_by_question.json", k))
                question_answer_metric.append(compute_prec(file_path + "/es_query_by_question_answer.json", k))
                question_answer_concat_metric.append(compute_prec(file_path + "/es_query_by_question_answer_concat.json", k))

            # compute MAP
            if i == 1:
                metric_name.append("MAP" + str(k))

            answer_metric.append(compute_map(file_path + "/es_query_by_answer.json"))
            question_metric.append(compute_map(file_path + "/es_query_by_question.json"))
            question_answer_metric.append(compute_map(file_path + "/es_query_by_question_answer.json"))
            question_answer_concat_metric.append(compute_map(file_path + "/es_query_by_question_answer_concat.json"))
            

            output[method + "_answer"] = answer_metric
            output[method + "_question"] = question_metric
            output[method + "_question_answer"] = question_answer_metric
            output[method + "_question_answer_concat"] = question_answer_concat_metric

        else:
        # Compute metrics for the supervised method
            for loss in loss_type:
                for query_type in query_types:
                    
                    for neg_type in neg_types:
                        
                        file_path = rank_results_filepath + "/" + ranker + "/" + loss + "/" + query_type + "/" + neg_type
                        
                        if os.path.isdir(file_path):

                            method = loss + "_" + query_type + "_" + neg_type

                            answer_metric = []
                            question_metric = []
                            question_answer_metric = []
                            question_answer_concat_metric = []

                            for k in topk:

                                # compute NDCG@k
                                answer_metric.append(compute_ndcg(file_path + "/reranked_query_by_answer.json", k))
                                question_metric.append(compute_ndcg(file_path + "/reranked_query_by_question.json", k))
                                question_answer_metric.append(compute_ndcg(file_path + "/reranked_query_by_question_answer.json", k))
                                question_answer_concat_metric.append(compute_ndcg(file_path + "/reranked_query_by_question_answer_concat.json", k))

                                # compute P@k
                                answer_metric.append(compute_prec(file_path + "/reranked_query_by_answer.json", k))
                                question_metric.append(compute_prec(file_path + "/reranked_query_by_question.json", k))
                                question_answer_metric.append(compute_prec(file_path + "/reranked_query_by_question_answer.json", k))
                                question_answer_concat_metric.append(compute_prec(file_path + "/reranked_query_by_question_answer_concat.json", k))

                            # compute MAP
                            answer_metric.append(compute_map(file_path + "/reranked_query_by_answer.json"))
                            question_metric.append(compute_map(file_path + "/reranked_query_by_question.json"))
                            question_answer_metric.append(compute_map(file_path + "/reranked_query_by_question_answer.json"))
                            question_answer_concat_metric.append(compute_map(file_path + "/reranked_query_by_question_answer_concat.json"))
                            

                            output[method + "_answer"] = answer_metric
                            output[method + "_question"] = question_metric
                            output[method + "_question_answer"] = question_answer_metric
                            output[method + "_question_answer_concat"] = question_answer_concat_metric
                        else:
                            pass
                        
    data = dict()
    methods = []
    for method in output:
        metrics = output[method]
        metrics = ["{0:.4f}".format(x) for x in metrics]
        data[method] = metrics
        methods.append(method)

    # Generate evaluation DataFrame 
    df = pd.DataFrame.from_dict(data, orient='index')
    df.columns = ["NDCG@3", "NDCG@5","NDCG@10", "P@3",	"P@5", "P@10", "MAP"]
    df['Method'] = methods
    df = df[["Method", "NDCG@3", "NDCG@5","NDCG@10", "P@3",	"P@5", "P@10", "MAP"]]
    df.reset_index(drop=True, inplace=True)
    return df

In [5]:
# generate evaluation dataframe
rank_results_filepath="../../../BERT-FAQ/data/StackFAQ/rank_results"
df = get_evaluation_df(rank_results_filepath)

In [6]:
df

,Method,NDCG@3,NDCG@5,NDCG@10,P@3,P@5,P@10,MAP
0,unsupervised_answer,0.6012,0.3608,0.6269,0.2865,0.6427,0.1946,0.3972
1,unsupervised_question,0.5803,0.5645,0.5923,0.5039,0.6222,0.3468,0.6648
2,unsupervised_question_answer,0.6803,0.5161,0.7067,0.4524,0.7253,0.3196,0.5893
3,unsupervised_question_answer_concat,0.7615,0.6138,0.7831,0.5284,0.7893,0.3637,0.6366
4,triplet_faq_simple_answer,0.7713,0.5426,0.7859,0.4484,0.7839,0.3081,0.6472
5,triplet_faq_simple_question,0.7582,0.7128,0.7707,0.6360,0.7928,0.4354,0.7605
6,triplet_faq_simple_question_answer,0.8250,0.6768,0.8372,0.5986,0.8400,0.4160,0.7436
7,triplet_faq_simple_question_answer_concat,0.8511,0.7134,0.8578,0.6224,0.8578,0.4284,0.7593
8,triplet_faq_hard_answer,0.7863,0.6066,0.7998,0.4967,0.8022,0.3198,0.7066
9,triplet_faq_hard_question,0.7473,0.7040,0.7628,0.6296,0.7800,0.4267,0.7527


In [7]:
# dump StackFAQ results to csv file
df.to_csv(rank_results_filepath + "/results.csv", index=False)